In [1]:
import csv

In [3]:
# read in control case pairs
case2ctrl = {}
with open('/Users/campbelle2/Documents/Sensitivity_analysis/sensitivity_analysis_pairs.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader) # skip header
        for line in reader:
            case2ctrl[line[0]]=line[1]

In [4]:
with open('/Users/campbelle2/Documents/Sensitivity_analysis/SPADE_output_case.csv', 'r') as f: 
    reader = csv.reader(f)
    next(reader) # skip header
    patterns = [l[0] for l in reader]

In [5]:
def load_patients(file_name):
    edc_info = {}
    with open(file_name, 'r') as f:
        # file rows are
        #person_id,Timing_Class,times,EDCs
        #4239598,1,2,"1-NUR04, 1-PSY17"
        reader = csv.reader(f)
        next(reader) #skip header
        for line in reader:
            pid = line[0]
            timing = int(line[1])-1
            edcs = [_.strip() for _ in line[3].split(",")]
            if pid not in edc_info:
                edc_info[pid] = [[], [], []]
            edc_info[pid][timing] = edcs
    return edc_info

In [7]:
case_edc_info = load_patients('/Users/campbelle2/Documents/Sensitivity_analysis/case_input.csv')

In [8]:
ctrl_edc_info = load_patients('/Users/campbelle2/Documents/Sensitivity_analysis/control_input.csv')

In [9]:
mcnemar_counts = {}
missing_cases = []
missing_ctrls = []
for pattern in patterns:
    mcnemar_counts[pattern] = [0, 0, 0, 0]
    pattern_edcs = [_.strip() for _ in pattern.split(",")]
    for case_pid, ctrl_pid in case2ctrl.items():
        if case_pid not in case_edc_info:
            missing_cases.append(case_pid)
            case_edc_info[case_pid] = [[], [], []]
        if ctrl_pid not in ctrl_edc_info:
            missing_ctrls.append(ctrl_pid)
            ctrl_edc_info[ctrl_pid] = [[], [], []]
        b_case_has_pattern = True
        b_ctrl_has_pattern = True    
        case_visit_info = case_edc_info[case_pid]
        ctrl_visit_info = ctrl_edc_info[ctrl_pid]
        for edc in pattern_edcs:
            tpl_idx = int(edc[0])-1
            b_case_has_pattern = b_case_has_pattern and (edc in case_visit_info[tpl_idx])
            b_ctrl_has_pattern = b_ctrl_has_pattern and (edc in ctrl_visit_info[tpl_idx])
        if b_case_has_pattern and b_ctrl_has_pattern:
            jdx = 0
        elif b_case_has_pattern and (not b_ctrl_has_pattern):
            jdx = 1
        elif (not b_case_has_pattern) and b_ctrl_has_pattern:
            jdx = 2
        else:
            jdx = 3
        mcnemar_counts[pattern][jdx] = mcnemar_counts[pattern][jdx] + 1
missing_cases = list(set(missing_cases))
missing_ctrls = list(set(missing_ctrls))

In [10]:
mcnemar_counts

{'1-ADM05': [30, 993, 970, 27941],
 '1-ALL03': [132, 1672, 1630, 26500],
 '1-ALL03,1-ALL04': [21, 678, 615, 28620],
 '1-ALL03,1-ALL04,2-ALL04': [8, 430, 322, 29174],
 '1-ALL03,2-ALL03': [13, 600, 463, 28858],
 '1-ALL03,2-ALL03,2-ALL04': [2, 311, 207, 29414],
 '1-ALL03,2-ALL04': [12, 566, 406, 28950],
 '1-ALL03,2-ALL04,3-ALL04': [4, 323, 241, 29366],
 '1-ALL04': [333, 2996, 2355, 24250],
 '1-ALL04,2-ALL03': [6, 633, 398, 28897],
 '1-ALL04,2-ALL03,2-ALL04': [4, 542, 331, 29057],
 '1-ALL04,2-ALL03,2-ALL04,3-ALL04': [2, 319, 221, 29392],
 '1-ALL04,2-ALL03,3-ALL04': [2, 358, 239, 29335],
 '1-ALL04,2-ALL04': [111, 1832, 1212, 26779],
 '1-ALL04,2-ALL04,2-NUT03': [0, 332, 0, 29602],
 '1-ALL04,2-ALL04,3-ALL03': [4, 331, 257, 29342],
 '1-ALL04,2-ALL04,3-ALL04': [45, 1090, 758, 28041],
 '1-ALL04,2-NUT03': [0, 455, 0, 29479],
 '1-ALL04,2-SKN02': [4, 330, 206, 29394],
 '1-EAR01': [526, 3054, 2969, 23385],
 '1-EAR01,1-EAR08': [9, 666, 365, 28894],
 '1-EAR01,1-EAR11': [19, 504, 617, 28794],
 '1-EAR01

In [11]:
missing_cases

[]

In [12]:
missing_ctrls

[]

In [13]:
len(mcnemar_counts)

234

In [14]:
with open('mc_nemar_counts.csv','w+') as f:
    for pattern, counts in mcnemar_counts.items():
        line = '\"{0}\"'.format(pattern)
        for c in counts:
            line = "{0},{1}".format(line, c)
        f.write("{0}\n".format(line))